**Содержание тетради**
- Импорт данных из таблицы `clean_flats`
- Разделение признаков по категориям
- Разбиение датасета на тренировочную и тестовую выборки
- Обработка признаков с помощью пайплайна
- Обучение модели на тренировочной выборке и проведение кросс-валидации
- Оценка качества модели на тестовой выборке

In [14]:
# настройка загрузки расширений и модулей

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
# импорт библиотек и настройка параметров

import os
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv

from sklearn.metrics import root_mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split, KFold, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from catboost import CatBoostRegressor

pd.set_option('display.float_format', '{:,.2f}'.format)
RANDOM_STATE = 42

In [16]:
# определение вспомогательных функций для импорта данных и вывода статистик

def create_connection():
    load_dotenv()
    host = os.environ.get('DB_DESTINATION_HOST')
    port = os.environ.get('DB_DESTINATION_PORT')
    db = os.environ.get('DB_DESTINATION_NAME')
    username = os.environ.get('DB_DESTINATION_USER')
    password = os.environ.get('DB_DESTINATION_PASSWORD')
    conn = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{db}')
    return conn

def display_statistics(data, freq_values=True, decimals=2):
    if freq_values:
        freq_name = 'freq_values'
        freq_num = 3
    else:
        freq_name = 'most_freq'
        freq_num = 1
    return pd.DataFrame(
        {'type': [data[x].dtypes for x in data.columns],
         'count' : [data[x].count() for x in data.columns],
         'NaNs' : [data[x].isna().sum() for x in data.columns],
         'zero_values': [data[x].eq(0).sum() for x in data.columns],
         'unique_values': [data[x].nunique() for x in data.columns],
         freq_name: [data[x].round(decimals).value_counts().head(freq_num).to_dict() for x in data.columns],
         'min': [data[x].min() if data[x].dtype!=object else '---' for x in data.columns],
         'mean': [data[x].mean() if data[x].dtype!=object else '---' for x in data.columns],
         'max': [data[x].max() if data[x].dtype!=object else '---' for x in data.columns],
         'std': [data[x].std() if data[x].dtype!=object else '---' for x in data.columns],
         'lo_count': [lo_hi_count(data, x) for x in data.columns],
         'hi_count': [lo_hi_count(data, x, low=False) for x in data.columns],
        }, index = data.columns)

def lo_hi_count(data, col, low=True):
    if data[col].dtype not in [float, int,'datetime64[ns]']:
        return '---'
    Q1 = np.nanquantile(data[col], 0.25)
    Q3 = np.nanquantile(data[col], 0.75)
    if low:
        return data[data[col] <= (Q1 - 1.5 * (Q3 - Q1))][col].count()
    else:
        return data[data[col] >= (Q3 + 1.5 * (Q3 - Q1))][col].count()

In [17]:
conn = create_connection()
data = pd.read_sql('select * from clean_flats', conn).drop(['id', 'flat_id'], axis=1)
display_statistics(data)

,type,count,NaNs,zero_values,unique_values,freq_values,min,mean,max,std,lo_count,hi_count
floor,int64,130755,0,0,44,"{2: 13606, 3: 12656, 5: 11812}",1.00,7.42,44.00,5.55,0,3779
is_apartment,int64,130755,0,129546,2,"{0: 129546, 1: 1209}",0.00,0.01,1.00,0.10,129546,130755
kitchen_area,float64,130755,0,0,2627,"{6.0: 14581, 10.0: 12389, 9.0: 8907}",2.90,10.11,70.00,5.17,0,9584
living_area,float64,130755,0,0,3965,"{19.0: 5639, 20.0: 4559, 30.0: 3601}",10.00,36.08,230.00,20.85,0,5459
rooms,int64,130755,0,0,5,"{2: 49303, 1: 38906, 3: 34205}",1.00,2.11,5.00,0.94,0,0
total_area,float64,130755,0,0,3061,"{38.0: 3337, 45.0: 2625, 39.0: 2308}",19.90,60.73,349.00,32.69,0,6907
price,int64,130755,0,0,7744,"{10500000: 2143, 9500000: 1959, 12500000: 1844}","70,000.00","17,456,666.65","336,000,000.00","22,537,206.42",0,12966
building_id,int64,130755,0,0,24387,"{24195: 516, 24035: 212, 24057: 139}",1.00,"14,003.95","24,620.00","6,960.57",0,0
build_year,int64,130755,0,0,118,"{2017: 4071, 2018: 3973, 1968: 3272}","1,901.00","1,986.46","2,023.00",21.99,623,0
building_type_int,int64,130755,0,1726,6,"{4: 73635, 2: 22764, 1: 21269}",0.00,3.25,6.00,1.46,0,0


In [18]:
data.shape

(130755, 16)

In [6]:
cat_features= ['is_apartment', 'has_elevator','rooms', 'building_type_int']
target = ['price']
numeric_cat_features = [x for x in data.columns if x not in cat_features+target]

In [7]:
# в категориальных признаках наблюдается дисбаланс
# для базовой модели оставляем как есть

for x in cat_features:
    print(x, data[x].value_counts().to_dict())

is_apartment {0: 129546, 1: 1209}
has_elevator {1: 117402, 0: 13353}
rooms {2: 49303, 1: 38906, 3: 34205, 4: 6398, 5: 1943}
building_type_int {4: 73635, 2: 22764, 1: 21269, 6: 9915, 0: 1726, 3: 1446}


In [8]:
X_train, X_test, y_train, y_test = train_test_split(data.drop(target, axis=1), data[target], random_state=RANDOM_STATE) 

In [9]:
# используем стандартный метод для номинальных переменных OneHotEncoder
# во избежание dummy trap указываем drop

preprocessor = ColumnTransformer(
    [
    ('one_hot_drop', OneHotEncoder(drop='first', sparse_output=False), cat_features),
    ],
    remainder='passthrough',
    verbose_feature_names_out=False
)

encoded_data = pd.DataFrame(preprocessor.fit_transform(data), columns=preprocessor.get_feature_names_out())
other_features = [x for x in encoded_data.columns if x not in numeric_cat_features+target]
display_statistics(encoded_data[other_features])


In [10]:
# воспользуемся CatBoostRegressor с параметрами, указанными разработчиками для тестирования алгоритма
# в качестве лосс-функции укажем MAE с меньшей чувствительностью к выбросам
# применим кросс-валидацию с перемешиванием

model = CatBoostRegressor(learning_rate=1, 
                          depth=6,
                          loss_function='MAE', 
                          random_seed=RANDOM_STATE, 
                          verbose=0)
pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])
pipeline.fit(X_train, y_train)

cv_strategy = KFold(n_splits=5, random_state=RANDOM_STATE, shuffle=True)
cv_res = cross_validate(pipeline, 
                        X_train, 
                        y_train, 
                        cv=cv_strategy, 
                        scoring=('neg_mean_absolute_error', 'neg_root_mean_squared_error'), 
                        n_jobs=-1, 
                        verbose=2)
for key, value in cv_res.items():
    print(f'avg_{key}: {value.mean() if key.find("neg")<0 else -value.mean() :,.2f}')

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


[CV] END .................................................... total time=  49.9s
[CV] END .................................................... total time=  51.1s
[CV] END .................................................... total time=  51.4s
[CV] END .................................................... total time=  50.3s
[CV] END .................................................... total time=  20.9s
avg_fit_time: 44.29
avg_score_time: 0.45
avg_test_neg_mean_absolute_error: 3,636,462.00
avg_test_neg_root_mean_squared_error: 9,397,433.07


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  2.1min finished


In [11]:
# проверка на тестовой выборке

y_pred = pipeline.predict(X_test)
print(f"Mean Squared Error: {mean_absolute_error(y_test, y_pred):,.2f}")
print(f"Root Mean Squared Error: {root_mean_squared_error(y_test, y_pred):,.2f}")

Mean Squared Error: 3,436,851.93
Root Mean Squared Error: 8,667,899.69


**Выводы**    

- импорт данных проведен успешно
- выделены категориальные, в т.ч. бинарные, признаки
- выбраны методы кодирования и стандартизации, объединенные в пайплайн предобработки
- выбран алгоритм обучения
- проведена кросс-валидация на 5 фолдах 
- проведена оценка модели на тестовой выборке, переобучение модели не отмечено
- переобучение не отмечено, можно проводить предсказания цены с точностью +/- 3.6 млн руб
- создан минимально жизнеспособный продукт модели